In [1]:
import os
path = os.path.normpath(os.getcwd() + os.sep + os.pardir)
os.chdir(path)
path

'E:\\Semester_10\\IE506\\KFSC-Python'

In [2]:
!pip install soyclustering

In [3]:
import time
from multiprocessing import Pool
import multiprocessing
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import accuracy_score
from BestMap import BestMap
from KFSC import KFSC
from KFSC_LARGE import KFSC_LARGE
from KFSC_minibatch import KFSC_MB
from multi_tester import test_dataset

C:\Users\anura\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\anura\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\anura\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
# dataset = 'mnist_sc_f150.mat'
# dataset = 'fmnist_fea_150.mat'
dataset = 'covtype.mat'
# dataset = 'PokerHand2.mat'
# dataset = 'Postures_m.mat'
# dataset = 'Epileptic.mat'
f = sio.loadmat(dataset)
X = f['X']
label = np.concatenate(f['Label'])
k = len(np.unique(label))
cpu_count = multiprocessing.cpu_count()
use_cpu = min(23,cpu_count) 
print("CPU count: ", cpu_count)
print("Using CPU: ", use_cpu)

CPU count:  8
Using CPU:  8


In [5]:
iterations = 23
funct_to_run = KFSC
params = {}
params["opt"] = {
    'solver': 2,
    'maxiter': 300,
    'tol': 1e-4,
    'init_type': 'k-means-cos',
    'nrep_kmeans': 1,
    'classifier': 're'
}
# params["opt"]['batch_size'] = 5*np.logspace(1, 3, iterations, dtype=int)
# params["opt"]['np'] = 4*np.ones((iterations), dtype=int)

In [6]:
# params["opt"]['batch_size']
# params["opt"]['np']

In [7]:
# [] + [X, k, label, funct_to_run, dataset]

In [8]:
arg_list = []
for i in range(iterations):
    arg_list.append([])
    arg_list[-1] += [X, k, label, funct_to_run, dataset]
    print(arg_list[-1])
    build_dict = {'lamda':0.4}
#     for key in params.keys():
#         build_dict[key] = params[key][i]
    build_dict['opt'] = {}
    for key in params['opt'].keys():
        if key == 'np':         
            build_dict['opt'][key] = params['opt'][key][i]
        else:
            build_dict['opt'][key] = params['opt'][key]
            
    arg_list[-1].append(build_dict)
    arg_list[-1] = tuple(arg_list[-1])
arg_list[20]

[array([[2596, 2590, 2804, ..., 2386, 2384, 2383],
       [  51,   56,  139, ...,  159,  170,  165],
       [   3,    2,    9, ...,   17,   15,   13],
       ...,
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int16), 7, array([5, 5, 2, ..., 3, 3, 3], dtype=uint8), <function KFSC at 0x0000018B59EE44C8>, 'covtype.mat']
[array([[2596, 2590, 2804, ..., 2386, 2384, 2383],
       [  51,   56,  139, ...,  159,  170,  165],
       [   3,    2,    9, ...,   17,   15,   13],
       ...,
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int16), 7, array([5, 5, 2, ..., 3, 3, 3], dtype=uint8), <function KFSC at 0x0000018B59EE44C8>, 'covtype.mat']
[array([[2596, 2590, 2804, ..., 2386, 2384, 2383],
       [  51,   56,  139, ...,  159,  170,  165],
       [   3,    2,    9, ...,   17,   

(array([[2596, 2590, 2804, ..., 2386, 2384, 2383],
        [  51,   56,  139, ...,  159,  170,  165],
        [   3,    2,    9, ...,   17,   15,   13],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0]], dtype=int16),
 7,
 array([5, 5, 2, ..., 3, 3, 3], dtype=uint8),
 <function KFSC.KFSC(X, k, d, lamda, options: Union[dict, NoneType] = None)>,
 'covtype.mat',
 {'lamda': 0.4,
  'opt': {'solver': 2,
   'maxiter': 300,
   'tol': 0.0001,
   'init_type': 'k-means-cos',
   'nrep_kmeans': 1,
   'classifier': 're'}})

In [ ]:
with Pool(processes=use_cpu) as p:
    df_list = p.starmap(test_dataset, arg_list)
combined_df = pd.concat(df_list, ignore_index=True)
path = os.getcwd()
os.chdir(path+'//Scores')
combined_df.to_csv(dataset.split('_')[0] + 'bulk_run_info.csv', mode='a', index=False, header=False)

In [ ]:
combined_df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pylab as pylab

params = {'legend.fontsize': 'large',
          'figure.figsize': (10, 7),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'figure.titlesize':'xx-large'}
pylab.rcParams.update(params)

In [ ]:
# xvals = [combined_df['parameters'][i]['nsel'] for 
#          i in range(len(list(combined_df['parameters'])))]
# xvals = [combined_df['parameters'][i]['opt']['batch_size'] for 
#          i in range(len(list(combined_df['parameters'])))]
xvals = np.arange(combined_df.shape[0])
plt.scatter(xvals, combined_df['acc'])
plt.ylabel("Accuracy"); plt.xlabel("Parameter")
# plt.xscale('log')
plt.grid(which='both')
plt.show()

plt.scatter(xvals, combined_df['nmi'])
plt.ylabel("Mutual Info"); plt.xlabel("Parameter")
plt.xscale('log')
plt.grid(which='both')
plt.show()